# Montando Ambiente

In [2]:
import sys

if 'google.colab' in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/
  %mkdir -p CasVit
  %cd CasVit

In [2]:
"""Descomentar caso seja primeira vez usando notebook"""
#!git clone https://github.com/johnnyharanaka/mmdetection.git
#!git clone https://github.com/johnnyharanaka/CAS-ViT.git

if 'google.colab' in sys.modules:
  %cd mmdetection
  !pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121
  !pip -q install openmim && mim install mmengine mmcv && pip install -e .
  !pip install "mmsegmentation>=1.0.0" && pip install timm==1.0.9
  !pip install tensorboardX
  %cd ..


In [3]:
if 'google.colab' in sys.modules:
  HOME = '/content/drive/MyDrive'
else:
  HOME = '/Users/haranaka/Development/AI/RedesNeurais'
"""
if 'google.colab' in sys.modules:
  !pip install gdown
  import gdown
  # Link do arquivo
  #Caltech-UCSD Birds-200-2011
  url = 'https://drive.google.com/uc?id=1mEKi61CNbla7KonRU1UYtN6NS9f5yHvc'
  url2 = 'https://drive.google.com/uc?id=1bGuV5-gR2gSQOiaSB3T5zvTRBEFe1UTd'

  # Caminho para salvar no Google Drive
  output = '/content/drive/MyDrive/CasVit/CAS-ViT/classification/data/cub100.zip'
  output = '/content/drive/MyDrive/CasVit/CAS-ViT/classification/data/meuDataset.zip'

  # Fazer o download
  gdown.download(url, output, quiet=False)
  gdown.download(url2, output2, quiet=False)
  !unzip -q /content/drive/MyDrive/CasVit/CAS-ViT/classification/data/cub100.zip -d /content/drive/MyDrive/CasVit/CAS-ViT/classification/data/
  !unzip -q /content/drive/MyDrive/CasVit/CAS-ViT/classification/data/meuDataset.zip -d /content/drive/MyDrive/CasVit/CAS-ViT/classification/data/
"""

"\nif 'google.colab' in sys.modules:\n  !pip install gdown\n  import gdown\n  # Link do arquivo\n  url = 'https://drive.google.com/uc?id=19l52ua_vvTtttgVRziCZJjal0TPE9f2p'\n\n\n  # Caminho para salvar no Google Drive\n  output = '/content/drive/MyDrive/CasVit/CAS-ViT/classification/data/data.zip'\n\n  # Fazer o download\n  gdown.download(url, output, quiet=False)\n  !unzip -q /content/drive/MyDrive/CasVit/CAS-ViT/classification/data/data.zip -d /content/drive/MyDrive/CasVit/CAS-ViT/classification/data/\n"

In [1]:
%cd CAS-ViT/classification

/Users/haranaka/Development/AI/RedesNeurais/CAS-ViT/classification


In [2]:
import torch

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

# Adapters

```python
class Adapter(nn.Module):
    def __init__(self,
                 d_model,
                 bottleneck,
                 dropout=0.2,
                 init_option="bert",
                 adapter_scalar="1.0",
                 adapter_layernorm_option="out"):
        super().__init__()

        self.n_embd = d_model
        self.down_size = bottleneck
        
        self.adapter_layernorm_option = adapter_layernorm_option

        self.adapter_layer_norm_before = None
        if adapter_layernorm_option == "in" or adapter_layernorm_option == "out":
            #self.adapter_layer_norm_before = nn.InstanceNorm2d(self.n_embd,affine=True)
            self.adapter_layer_norm_before = nn.BatchNorm2d(self.n_embd)

        if adapter_scalar == "learnable_scalar":
            self.scale = nn.Parameter(torch.ones(1))
        else:
            self.scale = float(adapter_scalar)

        self.down_proj = nn.Conv2d(self.n_embd, self.down_size, kernel_size=1, stride=1, padding=0)
        self.non_linear_func = nn.ReLU()

        #self.up_proj = nn.ConvTranspose2d(self.down_size, self.n_embd, kernel_size=3, stride=1, padding=1)
        self.up_proj = nn.Conv2d(self.down_size, self.n_embd, kernel_size=1, stride=1, padding=0)
        #self.up_proj = nn.Conv2d(self.down_size, self.n_embd, kernel_size=3, stride=1, padding=1)

        self.dropout = dropout
        if init_option == "bert":
            raise NotImplementedError
        elif init_option == "lora":
            with torch.no_grad():
                nn.init.kaiming_uniform_(self.down_proj.weight, a=math.sqrt(5))
                nn.init.zeros_(self.up_proj.weight)
                nn.init.zeros_(self.down_proj.bias)
                nn.init.zeros_(self.up_proj.bias)

    def forward(self, x, add_residual=False, residual=None):

        residual = x if residual is None else residual
        if self.adapter_layernorm_option == 'in':
            x = self.adapter_layer_norm_before(x)

        down = self.down_proj(x)
        down = self.non_linear_func(down)
        down = nn.functional.dropout(down, p=self.dropout, training=self.training)
        up = self.up_proj(down)

        up = up * self.scale
        
        if self.adapter_layernorm_option == 'out':
            up = self.adapter_layer_norm_before(up)

        if add_residual:
            output = up + residual
        else:
            output = up
        
        return output

<h1>Arquitetura do CAS-Vit</h1>
<img src="https://github.com/Tianfang-Zhang/CAS-ViT/blob/main/assets/arch.png?raw=true" alt="Exemplo de Imagem" width="600">


# Treino dos Modelos no Meu dataset

In [3]:
!python main.py --batch_size 32 --epochs 20 --model rcvit_xs --data_path data/meuDataset/ --lr 5e-5 --min_lr 1e-5 --weight_decay 0.05 --nb_classes 3 --output_dir data/res_fine --finetune model/cas-vit-xs.pth --warmup_epochs 3 --data_set image_folder --model_ema_eval False --model_ema False --device {device} --adapter False --input_size 384

Not using distributed mode
Number of the class = 3
Warping 384 size input images...
Number of the class = 3
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x3272e8e50>
log writter dir:  None
size mismatch for head.weight: copying a param with shape torch.Size([1000, 220]) from checkpoint, the shape in current model is torch.Size([3, 220]).
size mismatch for head.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([3]).
Finetune resume checkpoint: model/cas-vit-xs.pth
number of params: 2759363
LR = 0.00005000
Batch size = 64
Update frequent = 2
Number of training examples = 595
Number of training training per epoch = 9
/Users/haranaka/.pyenv/versions/3.9.6/envs/RN/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Use Cosine LR scheduler
Set warmup steps = 27
epochs: 20, niter_per_ep: 

In [115]:
!python main.py --batch_size 32 --epochs 20 --model rcvit_xs --data_path data/meuDataset/ --nb_classes 3 --lr 5e-4 --min_lr 1e-5 --weight_decay 0.5 --output_dir data/res --finetune model/cas-vit-xs.pth --warmup_epochs 1 --data_set image_folder  --adapter True --device {device}   --model_ema_eval False --model_ema False --input_size 384

Not using distributed mode
Number of the class = 3
Warping 384 size input images...
Number of the class = 3
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x324c9ce20>
log writter dir:  None
Weights of RCViT_adapter not initialized from pretrained model: ['cur_adapter.0.adapter_layer_norm_before.weight', 'cur_adapter.0.adapter_layer_norm_before.bias', 'cur_adapter.0.adapter_layer_norm_before.running_mean', 'cur_adapter.0.adapter_layer_norm_before.running_var', 'cur_adapter.0.down_proj.weight', 'cur_adapter.0.down_proj.bias', 'cur_adapter.0.up_proj.weight', 'cur_adapter.0.up_proj.bias', 'cur_adapter.1.adapter_layer_norm_before.weight', 'cur_adapter.1.adapter_layer_norm_before.bias', 'cur_adapter.1.adapter_layer_norm_before.running_mean', 'cur_adapter.1.adapter_layer_norm_before.running_var', 'cur_adapter.1.down_proj.weight', 'cur_adapter.1.down_proj.bias', 'cur_adapter.1.up_proj.weight', 'cur_adapter.1.up_proj.bias', 'cur_adapter.2.adapter_layer_norm_before.w

# Teste Dos Modelos no meu Dataset

In [35]:
!python main.py --batch_size 32 --model rcvit_xs --data_path data/meuDataset/ --nb_classes 3 --finetune data/res_fine/checkpoint-best*.pth  --data_set image_folder  --adapter False --device {device}  --test True

Not using distributed mode
Number of the class = 3
Finetune resume checkpoint: data/res_fine/checkpoint-best_224.pth
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        42
           1       1.00      0.93      0.96        40
           2       0.90      0.96      0.93        46

    accuracy                           0.94       128
   macro avg       0.94      0.94      0.94       128
weighted avg       0.94      0.94      0.94       128



In [117]:
!python main.py --batch_size 32 --model rcvit_xs --data_path data/meuDataset/ --nb_classes 3 --finetune data/res/checkpoint-best*.pth  --data_set image_folder  --adapter True --device {device}  --test True

Not using distributed mode
Number of the class = 3
Finetune resume checkpoint: data/res/checkpoint-best_384.pth
              precision    recall  f1-score   support

           0       0.98      0.95      0.96        42
           1       0.97      0.95      0.96        40
           2       0.96      1.00      0.98        46

    accuracy                           0.97       128
   macro avg       0.97      0.97      0.97       128
weighted avg       0.97      0.97      0.97       128



# CUB TREINO E TESTE

Treinamento com finetuning no Cas-Vit

In [38]:
!python main.py --batch_size 256 --epochs 100  --model rcvit_xs --data_path data/cub100/ --nb_classes 100 --lr 1e-4 --min_lr 5e-5 --weight_decay 0.5 --output_dir data/IMres_fine/ --finetune model/cas-vit-xs.pth --warmup_epochs 3 --data_set image_folder  --adapter False --device mps  --model_ema_eval False --model_ema False --disable_eval True --input_size 224

Not using distributed mode
Number of the class = 200
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x322597fd0>
log writter dir:  None
size mismatch for head.weight: copying a param with shape torch.Size([1000, 220]) from checkpoint, the shape in current model is torch.Size([200, 220]).
size mismatch for head.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([200]).
Finetune resume checkpoint: model/cas-vit-xs.pth
number of params: 2802900
LR = 0.01000000
Batch size = 512
Update frequent = 2
Number of training examples = 5981
Number of training training per epoch = 11
/Users/haranaka/.pyenv/versions/3.9.6/envs/RN/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Use Cosine LR scheduler
Set warmup steps = 33
epochs: 175, niter_per_ep: 11,  len(schedule): 1925
Set warmup steps = 0
e

Treinamento com Adapters

In [102]:
!python main.py --batch_size 256 --epochs 100  --model rcvit_xs --data_path data/cub100/ --nb_classes 100 --lr 1e-4 --min_lr 1e-5 --weight_decay 0.05 --output_dir data/cub_ada/ --finetune model/cas-vit-xs.pth --warmup_epochs 3 --data_set image_folder  --adapter True --device mps  --model_ema_eval False --model_ema False --disable_eval True --clip_grad 2.5 

Not using distributed mode
Number of the class = 100
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x16bc48f10>
log writter dir:  None
Weights of RCViT_adapter not initialized from pretrained model: ['cur_adapter.0.adapter_layer_norm_before.weight', 'cur_adapter.0.adapter_layer_norm_before.bias', 'cur_adapter.0.adapter_layer_norm_before.running_mean', 'cur_adapter.0.adapter_layer_norm_before.running_var', 'cur_adapter.0.down_proj.weight', 'cur_adapter.0.down_proj.bias', 'cur_adapter.0.up_proj.weight', 'cur_adapter.0.up_proj.bias', 'cur_adapter.1.adapter_layer_norm_before.weight', 'cur_adapter.1.adapter_layer_norm_before.bias', 'cur_adapter.1.adapter_layer_norm_before.running_mean', 'cur_adapter.1.adapter_layer_norm_before.running_var', 'cur_adapter.1.down_proj.weight', 'cur_adapter.1.down_proj.bias', 'cur_adapter.1.up_proj.weight', 'cur_adapter.1.up_proj.bias', 'cur_adapter.2.adapter_layer_norm_before.weight', 'cur_adapter.2.adapter_layer_norm_before.bias',

Teste do cub com Adapters

In [114]:
!python main.py --batch_size 256 --model rcvit_xs --data_path data/cub100/ --nb_classes 100 --finetune data/cub_ada/checkpoint-99.pth   --data_set image_folder  --adapter True --device {device}  --test True

Not using distributed mode
Number of the class = 100
Finetune resume checkpoint: data/cub_ada/checkpoint-99.pth
              precision    recall  f1-score   support

           0       0.41      0.65      0.50        17
           1       0.50      0.23      0.32        13
           2       1.00      0.15      0.27        13
           3       0.15      0.40      0.22         5
           4       0.00      0.00      0.00         8
           5       0.50      0.22      0.31         9
           6       0.22      0.62      0.32         8
           7       0.10      0.17      0.12         6
           8       0.33      0.10      0.15        10
           9       0.57      0.33      0.42        12
          10       0.50      0.11      0.18         9
          11       0.40      0.67      0.50         6
          12       0.35      0.67      0.46         9
          13       0.62      0.57      0.59        14
          14       0.46      0.60      0.52        10
          15       0.77

Teste do cub sem Adapters

In [98]:
!python main.py --batch_size 256 --model rcvit_xs --data_path data/cub100/ --nb_classes 100 --finetune data/cub_fine/checkpoint-99.pth  --data_set image_folder  --adapter False --device {device}  --test True

Not using distributed mode
Number of the class = 100
Finetune resume checkpoint: data/cub_fine/checkpoint-99.pth
              precision    recall  f1-score   support

           0       0.60      0.71      0.65        17
           1       0.62      0.38      0.48        13
           2       0.27      0.23      0.25        13
           3       0.36      0.80      0.50         5
           4       0.60      0.38      0.46         8
           5       0.86      0.67      0.75         9
           6       0.50      0.75      0.60         8
           7       0.30      0.50      0.38         6
           8       0.38      0.30      0.33        10
           9       0.67      0.67      0.67        12
          10       0.40      0.44      0.42         9
          11       0.83      0.83      0.83         6
          12       0.64      0.78      0.70         9
          13       0.87      0.93      0.90        14
          14       0.75      0.60      0.67        10
          15       1.0

# Resumo

***Meu Dataset*** :
Compostos por 3 classes.

- Parametros Treinaveis com Adapter:  134011

- Parametros Treinaveis sem Adapter: 2759363

- Diferenca: Adapter tem 4.85% de parametros em relacao a sem

***CUB100***: É o Dataset CUB com apenas 100 classes. Eu alterei, pois o dataset demorava muito para completar cada ciclo de treinamento. 

- Parametros Treinaveis com Adapter:  155448

- Parametros Treinaveis sem Adapter: 2802900

- Diferenca: Adapter tem 5.54% de parametros em relacao a sem

***Treinamento***
- Feito em 20 epocas com o meu dataset
- Feito em 100 epocas com o dataset CUB
- Apenas o treinamento de finetuning no CUB foi realizado no google colab, pois a maquina leva muito tempo.
- Resto do treinamento foi realizado em um MacBook com M2 pro.

***Link dos Datasets estao nas celulas anteriores***
-  url = 'https://drive.google.com/uc?id=1mEKi61CNbla7KonRU1UYtN6NS9f5yHvc'
-  url2 = 'https://drive.google.com/uc?id=1bGuV5-gR2gSQOiaSB3T5zvTRBEFe1UTd'

***Desempenho no Teste do Meu Dataset***
- Com Adapters teve um melhor desempenho, provavelmente pois o finetuning era mais complexo para generalizar.

***Desempenho no Teste do CUB***

- O Finetuning obteve um desempenho muito melhor em relacao aos adapters, valor saturou em 40% com Adapters.
- O modelo com Finetuning nao obter um valor maior ja era esperado, pois eu peguei o modelo mais simples.

<img src="https://github.com/sun-hailong/CVPR24-Ease/blob/main/resources/result-img.png?raw=true" alt="Exemplo de Imagem" width="900">


